In [6]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from os import listdir
import data_visualization as dv
import pickle
from scipy.stats import norm
from scipy import stats
from tqdm import tqdm 
'''
This script pulls
1. Decision array of agent and player
2. Decision time of agent and player
   - Possibly calculate this with velocity in the future
3. Control things
   - Reaction times, interval target reach times, coincidence target reach times
   
   
*****v2 GOES BACK TO THE REACTION WITH THE CIRCLES INSTEAD OF THE AGENT****
'''

'\nThis script pulls\n1. Decision array of agent and player\n2. Decision time of agent and player\n   - Possibly calculate this with velocity in the future\n3. Control things\n   - Reaction times, interval target reach times, coincidence target reach times\n   \n   \n*****v2 GOES BACK TO THE REACTION WITH THE CIRCLES INSTEAD OF THE AGENT****\n'

In [7]:
# Fields pull and pull list
os.chdir('D:\Subject_Data\Seth_MatchPennies_Agent_Exp1')
PATH = os.getcwd()
analysis_pull_list = []
analysis_pull_list_control = []
analysis_pull_list_task = []
        
with open(PATH+"\\Analysis_Pull_List.txt", "r") as file:
    analysis_pull_list = file.read().splitlines()
analysis_pull_list_control = [s+'_Control' for s in analysis_pull_list]
analysis_pull_list_task = [s+'_Task' for s in analysis_pull_list]

# with open(PATH+"\\Analysis_Pull_List_Control.txt", "r") as pull_file:
#     analysis_pull_list_control = pull_file.read().splitlines()
# with open(PATH+"\\Analysis_Pull_List_Task.txt", "r") as pull_file:
#     analysis_pull_list_task = pull_file.read().splitlines()
with open(PATH+"\\Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()
    
num_subjects = len(analysis_pull_list_control)
task_name = 'Seth_MatchPennies_Agent_Exp1'



Target information

- Row 0 = Start Target

- Row 1 = Left screen, right target x pos (left target is 2*startx - right targetx)

- Dim 1 = Radius

- Dim 2 = Thickness of the circle edge (don't know if this matters)


Dataframe is in centimeters, so need to divide everything by 100

In [8]:
PATH = r'D:\Subject_Data\Seth_MatchPennies_Agent_Exp1'

In [9]:
file = PATH+'\\Sub1_Task\\Sub1_TaskTarget_Table.csv'
df = pd.read_csv(file)
df["X"] = df["X"]/100
df["Y"] = df["Y"]/100
df['Dim 1'] = df['Dim 1']/100 # Target table is in centimeters, I guess this doesn't matter but it makes me feel better
df['Dim 2'] = df['Dim 2']/100
# Target information for Right Hand (keeping this because the positions of target 3 and 4 are based on target 1 and start 1)
startx = df.loc[0]['X']
starty = df.loc[0]['Y']
start_radius = df.loc[0]['Dim 1'] 

target1x = df.loc[1]['X']
target1y = df.loc[1]['Y']
target1_radius = df.loc[1]['Dim 1']

target2x = 2*startx - target1x
target2y = target1y
target2_radius = target1_radius

# Timing target
timing_targetx = startx
timing_targety = target1y
timing_target_pos = np.sqrt(timing_targetx**2 + timing_targety**2)
timing_target_radius = target1_radius   

Control Experiment Data

In [11]:
#%% Get reaction time data
reaction_trial_time = 5000
reaction_trials = 50
player_reaction_decision_array = np.empty((num_subjects, reaction_trials))
reaction_trial_start = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_movement_time = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_plus_movement_time = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_time = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_time_vel = np.zeros((num_subjects, reaction_trials))*np.nan
agent_reaction_decision_time = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_reach_time = np.zeros((num_subjects, reaction_trials))*np.nan
reaction_data = np.zeros((num_subjects, reaction_trials, reaction_trial_time, 2))*np.nan


coincidence_trial_time = 2500
coincidence_trials = 50
coincidence_trial_start = np.zeros((num_subjects, coincidence_trials))*np.nan
coincidence_reach_time = np.zeros((num_subjects, coincidence_trials))*np.nan
coincidence_data = np.zeros((num_subjects, coincidence_trials, coincidence_trial_time, 2))*np.nan

interval_trial_time = 2500
interval_trials = 50
interval_trial_start = np.zeros((num_subjects, interval_trials))*np.nan
interval_reach_time =  np.zeros((num_subjects, interval_trials))*np.nan
interval_data = np.zeros((num_subjects, interval_trials, interval_trial_time, 2))*np.nan

control_trials = interval_trials + coincidence_trials + reaction_trials
###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    sub_name = analysis_pull_list_control[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    control_trial_table = pd.read_csv(file)
    block_number = 1
    tp_num = 1
    for x in ((range(control_trials))):
        block_number = control_trial_table.iloc[x]['Block_Row']
        tp_num = control_trial_table.iloc[x]['TP_Row']
        block_trial_num = control_trial_table.iloc[x]['Block_Step']
        filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        j = tp_num - 1 # Block number
        k = block_trial_num - 1 # Trial number in that block
        data = pd.read_csv(filename, low_memory=False)
        # if 'Agent_Initial_Time' in data.columns:
        #     data = data.rename(columns = {'Agent_Inital_Time':'Agent_Decision_Time'}, inplace=True)
        # f = data['Agent_Decision_Time']
        start_time = int(data[data['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
        # Get Interval Timing Data
        if tp_num == 1:
            end_time = start_time + interval_trial_time
            interval_trial_start[i,k] = start_time # Store start time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                interval_reach_time[i,k] = q[0]
        # Get Coincidence Timing Data
        if tp_num == 2:
            end_time = start_time + coincidence_trial_time
            coincidence_trial_start[i,k] = start_time # Store start time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                coincidence_reach_time[i,k] = q[0]
        # Get Reaction Time Data
        if tp_num == 3:
            end_time = start_time + reaction_trial_time
            reaction_trial_start[i,k] = start_time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            lhx_vel = np.array(data.iloc[start_time:end_time]['Left_HandXVel'])
            lhy_vel = np.array(data.iloc[start_time:end_time]['Left_HandYVel'])
            q = np.argwhere(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2) < target1_radius)
            r = np.argwhere(np.sqrt((lhx-target2x)**2 + (lhy-target2y)**2) < target2_radius) 
            s = np.argwhere(np.sqrt((lhx-startx)**2 + (lhy-starty)**2) > start_radius*1.2) # Multiplied by 2 so people don't accidentally drift outside start
            
            t = np.argwhere(np.sqrt((lhx_vel)**2 + (lhy_vel)**2) > 0.05)
            if np.size(t)>0:
                player_reaction_time_vel[i,k] = t[0]
            if np.size(s)>0: 
                player_reaction_time[i,k] = s[0] # lhx and lhy are already constrained from start_time to end_time, so as soon as they leave the start, that's their reaction time
            if np.size(q)>0: #if LH enters right target
                player_reaction_decision_array[i,k] = 1
                player_reaction_movement_time[i,k] = q[0] - s[0]
                player_reaction_plus_movement_time[i,k] = q[0]
                
            elif np.size(r)>0: #if LH enters left target
                player_reaction_decision_array[i,k] = -1   
                player_reaction_movement_time[i,k] = r[0] - s[0]
                player_reaction_plus_movement_time[i,k] = r[0]                
                
            # #plot path for Reactions
            if False:
                for i in range(1):
                    player_reaction_time[0,0] = 500
                    plot_end_time = int(start_time + player_reaction_time[i,k])
                    # plot_end_time = int(start_time + 500)
                    lhx_new = np.array(data.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                    lhy_new = np.array(data.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                    plt.figure(dpi=300)
                    circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                    circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                    startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                    fig, ax = plt.subplots()
                    ax.add_patch(circleR)
                    ax.add_patch(circleL)
                    ax.add_patch(startCirc)
                    plt.plot(lhx_new,lhy_new) 
                    # plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])
                    print(player_reaction_time[i,k])
                    # plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                    plt.show()

           

0 Sub1_Control
1 Sub2_Control


In [ ]:
blah

In [ ]:
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    print(data_path)
    # Reaction time 
    pickle.dump(reaction_trial_start[i,:], open(data_path + f'{subname}_reaction_trial_start.pkl', 'wb'))
    print(player_reaction_time[i,:])
    pickle.dump(player_reaction_time[i,:], open(data_path + f'{subname}_player_reaction_time.pkl', 'wb'))
    pickle.dump(player_reaction_movement_time[i,:], open(data_path + f'{subname}_player_reaction_movement_time.pkl', 'wb'))
    pickle.dump(player_reaction_plus_movement_time[i,:], open(data_path + f'{subname}_player_reaction_plus_movement_time.pkl', 'wb'))
    pickle.dump(player_reaction_decision_array[i,:], open(data_path + f'{subname}_player_reaction_decision_array.pkl', 'wb'))
    pickle.dump(agent_reaction_decision_time[i,:], open(data_path + f'{subname}_agent_reaction_decision_time.pkl', 'wb'))
    pickle.dump(reaction_trial_start[i,:], open(data_path + f'{subname}_reaction_trial_start.pkl', 'wb'))
    pickle.dump(interval_trial_start[i,:], open(data_path + f'{subname}_interval_trial_start.pkl', 'wb'))
    pickle.dump(interval_reach_time[i,:], open(data_path + f'{subname}_interval_reach_time.pkl', 'wb'))
    pickle.dump(coincidence_trial_start[i,:], open(data_path + f'{subname}_coincidence_trial_start.pkl', 'wb'))
    pickle.dump(coincidence_reach_time[i,:], open(data_path + f'{subname}_coincidence_reach_time.pkl', 'wb'))


Sub1
D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\Sub1\
[555. 399. 276. 305. 301. 332. 321. 305. 342. 442. 231. 237. 247. 277.
 293. 277. 257. 289. 261. 277. 239. 304. 258. 279. 314. 248. 302. 246.
 291. 275. 263. 313. 263. 276. 251. 235. 261. 260. 243. 267. 226. 302.
 250. 242. 252. 269. 310. 289. 248. 268.]
Sub2
D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\Sub2\
[483. 315. 323. 243. 288. 283. 281. 288. 286. 311. 314. 292. 325. 273.
 262. 282. 274. 332. 331. 304. 259. 301. 304. 288. 264. 287. 319. 286.
 336. 303. 325. 325. 321. 260. 282. 303. 297. 309. 296. 290. 297. 314.
 299. 271. 296. 258. 278. 339. 301. 303.]
Sub3
D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\Sub3\
[371. 250. 310. 375. 337. 313. 290. 329. 295. 351. 293. 326. 261. 250.
 279. 310. 252. 316. 300. 322. 364. 331. 269. 295. 308. 281. 299. 330.
 439. 301. 311. 313. 377. 303. 397. 342. 337. 374. 280. 282. 314. 368.
 266. 278. 339. 270. 358. 339. 314. 383.]
Sub4
D:\Subje

Task Data 

In [ ]:
#%% Get task time task_df
# Get trials, blocks, trial_time from trial_table
path1 = PATH+'\\'+'Sub1_Task'
task_df = pd.read_csv(path1+f'\\Sub1_TaskTrial_Table.csv')
task_df = task_df.loc[task_df['Condition type']==3] # Only get the task condition 
num_trials = int(task_df.iloc[-1]['Block_Step']) # number of trials in each block
num_blocks = int(task_df.iloc[-1]['Block_Row']/2)
tot_trials = int(num_trials*num_blocks)
trial_time = int(task_df.iloc[0]['Condition time'])
task_df_columns = len(fields_pull)
# Need to be kept outside subject for-loop
task_trial_end = np.zeros([num_subjects, num_blocks, num_trials, 2])*np.nan
task_trial_start = np.zeros((num_subjects, num_blocks, num_trials))*np.nan
filename_storage = np.empty((num_subjects, num_blocks, num_trials), dtype=object)

task_data = np.zeros((num_subjects, num_blocks, num_trials, trial_time+500, task_df_columns))*np.nan
win_check_R = np.zeros((num_subjects, num_blocks))
win_check_L = np.zeros((num_subjects, num_blocks))
num_miss_check_R = np.zeros((num_subjects, num_blocks))
num_miss_check_L = np.zeros((num_subjects, num_blocks))
#########
player_task_decision_array = np.empty((num_subjects, num_blocks, num_trials))
player_task_reach_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_task_decision_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_task_movement_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
agent_task_decision_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_task_reach_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_task_decision_array = np.zeros((num_subjects, num_blocks, num_trials))

###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    sub_name = analysis_pull_list_task[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    trial_table = pd.read_csv(file)
    # Splt trial table into task and washout based on condition type
    task_trial_table = trial_table[trial_table['Condition type']==3]
    block_number = 1
    tp_num = 1
    for x in tqdm((range(tot_trials))):
        block_number = task_trial_table.iloc[x]['Block_Row']
        tp_num = task_trial_table.iloc[x]['TP_Row']
        block_trial_num = task_trial_table.iloc[x]['Block_Step']
        j = tp_num - task_trial_table['TP_Row'].min()  # Block number
        k = block_trial_num - 1 # Trial number in that block
        
        filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        task_df = pd.read_csv(filename, low_memory=False)
        start_time = int(task_df[task_df['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
        end_time = start_time + trial_time + 500 # Add 500 to see when they got there when trial ended
        data_task_df = task_df.drop(columns ='Event_Codes') # Drop event codes cuz it's not number column and can't be an array 
        task_data[i,j,k,:,:] = np.array(data_task_df.iloc[start_time:end_time])

        task_trial_start[i,j,k] = start_time
        lhx = np.array(task_df.iloc[start_time:end_time]['Left_HandX'])
        lhy = np.array(task_df.iloc[start_time:end_time]['Left_HandY'])
        q = np.argwhere(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2) < target1_radius)
        r = np.argwhere(np.sqrt((lhx-target2x)**2 + (lhy-target2y)**2) < target2_radius) 
        s = np.argwhere(np.sqrt((lhx-startx)**2 + (lhy-starty)**2) > start_radius*1.2) # Multiplied by 2 so people don't accidentally drift outside start
        
        #print(vel_check)
        agent_task_decision_time[i,j,k] = task_df.iloc[start_time+1]['Agent_Decision_Time'] 
        agent_task_reach_time[i,j,k] = agent_task_decision_time[i,j,k] + 150  
        agent_task_decision_array[i,j,k] = task_df.iloc[start_time+1]['Agent_Target_Selection']
        if np.size(s)>0: 
            vel_check = task_df.iloc[s[0]]['Left_HandYVel']
            player_task_decision_time[i,j,k] = s[0]
            
        if np.size(q)>0: #if LH enters right target
            player_task_decision_array[i,j,k] = 1
            player_task_movement_time[i,j,k] = q[0] - s[0]
            player_task_reach_time[i,j,k] = q[0]
        elif np.size(r)>0: #if LH enters left target
            player_task_decision_array[i,j,k] = -1   
            player_task_movement_time[i,j,k] = r[0] - s[0]
            player_task_reach_time[i,j,k] = r[0]
        if player_task_reach_time[i,j,k] > 1500 or np.isnan(player_task_reach_time[i,j,k]):
            player_task_decision_array[i,j,k] = 0
        if agent_task_reach_time[i,j,k]>1500:
            agent_task_decision_array[i,j,k] = 0    
        #plot path for tasks
        if False:
            for i in range(1):
                plot_end_time = int(start_time + player_task_reach_time[i,j,k])
                lhx_new = np.array(task_df.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                lhy_new = np.array(task_df.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                plt.figure(dpi=300)
                circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                fig, ax = plt.subplots()
                ax.add_patch(circleR)
                ax.add_patch(circleL)
                ax.add_patch(startCirc)
                plt.plot(lhx_new,lhy_new) 
                plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])

                plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                plt.show()
    print(player_task_decision_time[i,0,:])
           

0 Sub1_Task


100%|██████████| 480/480 [00:13<00:00, 35.46it/s]


[1219. 1138. 1030. 1225. 1342. 1264. 1245. 1320. 1104. 1202. 1280. 1085.
 1262. 1342. 1182. 1106. 1018. 1174. 1064.  999. 1116. 1151. 1056. 1050.
 1140.  972. 1104. 1196. 1137. 1213. 1225. 1309. 1199. 1131. 1250. 1137.
 1206. 1068. 1046. 1161. 1146. 1047. 1209. 1213. 1303. 1278. 1156. 1269.
 1239. 1213. 1298. 1219. 1324. 1267. 1304. 1358. 1233. 1237. 1185. 1091.
 1119. 1321. 1130. 1113. 1208. 1247. 1216. 1074. 1086. 1126. 1201. 1175.
 1156. 1325. 1252. 1173. 1191. 1243. 1306. 1090.]
1 Sub2_Task


100%|██████████| 480/480 [00:14<00:00, 32.83it/s]


[1274. 1120. 1048. 1139. 1242. 1154. 1184. 1126. 1222. 1123. 1259. 1273.
 1121. 1245. 1184. 1238. 1190. 1268. 1241. 1286. 1276. 1289. 1153. 1237.
 1256. 1262. 1273. 1275. 1383. 1235. 1343. 1358. 1343. 1221. 1322. 1292.
 1250. 1289. 1371. 1262. 1317. 1281. 1403. 1267. 1230. 1237. 1286. 1304.
 1243. 1286. 1301. 1282. 1276. 1327. 1190. 1279. 1221. 1380. 1357. 1295.
 1324. 1328. 1258. 1284. 1287. 1186. 1252. 1335. 1287. 1327. 1265. 1329.
 1268. 1312. 1343. 1326. 1319. 1259. 1278. 1330.]
2 Sub3_Task


...

[1322. 1324. 1194. 1246. 1228. 1090. 1237. 1081. 1375. 1190. 1021. 1020.
 1050. 1207. 1256. 1272. 1240. 1245. 1274. 1282. 1233. 1325. 1285. 1238.
 1234. 1289. 1244. 1297. 1292. 1292. 1268. 1283. 1309. 1356. 1257. 1141.
 1256. 1266. 1275. 1365. 1198. 1267. 1262. 1313. 1320. 1341. 1376. 1335.
 1255. 1199. 1250. 1262. 1211. 1342. 1318. 1236. 1310. 1260. 1206. 1289.
 1332. 1286. 1339. 1349. 1324. 1303. 1290. 1298. 1418. 1244. 1320. 1304.
 1247. 1353. 1279. 1339. 1300. 1308. 1322. 1344.]
18 Sub19_Task


100%|██████████| 480/480 [00:24<00:00, 19.79it/s]


[1301. 1297. 1243. 1301. 1317. 1246. 1245. 1291. 1294. 1229. 1289. 1277.
 1301. 1238. 1169. 1199. 1310. 1235. 1279. 1254. 1308. 1343. 1277. 1290.
 1274. 1294. 1275. 1207. 1293. 1312. 1259. 1314. 1330. 1285. 1178. 1283.
 1284. 1310. 1237. 1179. 1210. 1217. 1323. 1193. 1234. 1193. 1230. 1277.
 1204. 1282. 1259. 1269. 1251. 1261. 1272. 1260. 1298. 1242. 1274. 1258.
 1283. 1222. 1257. 1250. 1227. 1284. 1377. 1255. 1346. 1345. 1212. 1261.
 1269. 1333. 1257. 1300. 1291. 1365. 1342. 1249.]
19 Sub20_Task


100%|██████████| 480/480 [00:40<00:00, 11.95it/s]

[1234. 1074. 1005.  978. 1216. 1289. 1213.  932.  901. 1027. 1334. 1332.
 1040.  966. 1185.  894. 1091. 1299. 1277. 1160. 1177. 1201. 1283. 1236.
 1127. 1000. 1091. 1039. 1160. 1247. 1104. 1325. 1163. 1310. 1090.  826.
 1244. 1051.  983.  803. 1301. 1108. 1258.  975. 1148. 1094.  837. 1287.
 1042. 1155. 1345. 1195.  977.  996. 1053. 1061. 1094. 1070.  852. 1255.
 1108. 1298. 1274. 1287. 1261. 1268. 1121. 1244. 1276. 1128. 1321. 1221.
 1280. 1229. 1221. 1333. 1129. 1256. 1288. 1342.]


Pickle Task Data

In [ ]:
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    pickle.dump(task_data[i,:,:,:,:], open(data_path + f'{subname}_task_data.pkl', 'wb'))
    pickle.dump(player_task_decision_time[i,:,:], open(data_path + f'{subname}_player_task_decision_time.pkl', 'wb'))
    pickle.dump(player_task_decision_array[i,:,:], open(data_path + f'{subname}_player_task_decision_array.pkl', 'wb'))
    pickle.dump(player_task_movement_time[i,:,:], open(data_path + f'{subname}_player_task_movement_time.pkl', 'wb'))
    pickle.dump(player_task_reach_time[i,:,:], open(data_path + f'{subname}_player_task_reach_time.pkl', 'wb'))
    pickle.dump(agent_task_decision_time[i,:,:], open(data_path + f'{subname}_agent_task_decision_time.pkl', 'wb'))
    pickle.dump(agent_task_decision_array[i,:,:], open(data_path + f'{subname}_agent_task_decision_array.pkl', 'wb'))
    pickle.dump(agent_task_reach_time[i,:,:], open(data_path + f'{subname}_agent_task_reach_time.pkl', 'wb'))

Sub1
Sub2
Sub3
Sub4
Sub5
Sub6
Sub7
Sub8
Sub9
Sub10
Sub11
Sub12
Sub13
Sub14
Sub15
Sub16
Sub17
Sub18
Sub19
Sub20


Washout Data

In [ ]:
#%% Get washout time washout_df
# Get trials, blocks, trial_time from trial_table
path1 = PATH+'\\'+'Sub1_Task'
trial_table = pd.read_csv(path1+'\\Sub1_TaskTrial_Table.csv')
block_table = pd.read_csv(path1+'\\Sub1_TaskBlock_Table.csv')
washout_trial_table = trial_table[trial_table['Condition type']==4] # Only get the washout condition 
num_trials = int(washout_trial_table.iloc[-1]['Block_Step']) # number of trials in each block
num_blocks = pd.DataFrame(block_table[block_table['TP_LIST']==12]['TP_LIST'].value_counts()).iloc[0]['TP_LIST'] # Counts how many times TP_LIST is 12 (washout tp), and then gets that number with .iloc[0]['TP_LIST]
tot_trials = int(num_trials*num_blocks)
trial_time = int(washout_trial_table.iloc[0]['Condition time'])
washout_columns = len(fields_pull)
# Need to be kept outside subject for-loop
washout_trial_end = np.zeros([num_subjects, num_blocks, num_trials, 2])*np.nan
washout_trial_start = np.zeros((num_subjects, num_blocks, num_trials))*np.nan
filename_storage = np.empty((num_subjects, num_blocks, num_trials), dtype=object)

washout_data = np.zeros((num_subjects, num_blocks, num_trials, trial_time+500, washout_columns))*np.nan
win_check_R = np.zeros((num_subjects, num_blocks))
win_check_L = np.zeros((num_subjects, num_blocks))
num_miss_check_R = np.zeros((num_subjects, num_blocks))
num_miss_check_L = np.zeros((num_subjects, num_blocks))
#########
player_washout_decision_array = np.empty((num_subjects, num_blocks, num_trials))
player_washout_reach_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_washout_decision_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_washout_movement_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
agent_washout_decision_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_washout_reach_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_washout_decision_array = np.zeros((num_subjects, num_blocks, num_trials))

###-------------------------------------------------------------------------------------------------------
for i in (range(num_subjects)):
    sub_name = analysis_pull_list_task[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    trial_table = pd.read_csv(file)
    # Splt trial table into washout and washout based on condition type
    washout_trial_table = trial_table[trial_table['Condition type']==4]
    x = -1
    for j in (range(num_blocks)):
        for k in tqdm(range(num_trials)):
            x += 1
            block_number = washout_trial_table.iloc[x]['Block_Row']
            tp_num = washout_trial_table.iloc[x]['TP_Row']
            block_trial_num = washout_trial_table.iloc[x]['Block_Step']
            filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
            washout_df = pd.read_csv(filename, low_memory=False)
            start_time = int(washout_df[washout_df['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
            end_time = start_time + trial_time + 500 # Add 500 to see when they got there when trial ended
            data_washout_df = washout_df.drop(columns ='Event_Codes') # Drop event codes cuz it's not number column and can't be an array 
            washout_data[i,j,k,:,:] = np.array(data_washout_df.iloc[start_time:end_time])

            washout_trial_start[i,j,k] = start_time
            lhx = np.array(washout_df.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(washout_df.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2) < target1_radius)
            r = np.argwhere(np.sqrt((lhx-target2x)**2 + (lhy-target2y)**2) < target2_radius) 
            s = np.argwhere(np.sqrt((lhx-startx)**2 + (lhy-starty)**2) > start_radius*1.2) # Multiplied by 2 so people don't accidentally drift outside start
            
            #print(vel_check)
            agent_washout_decision_time[i,j,k] = washout_df.iloc[start_time+1]['Agent_Decision_Time']   
            agent_washout_decision_array[i,j,k] = washout_df.iloc[start_time+1]['Agent_Target_Selection']
            if np.size(s)>0: 
                vel_check = washout_df.iloc[s[0]]['Left_HandYVel']
                player_washout_decision_time[i,j,k] = s[0]
                
            if np.size(q)>0: #if LH enters right target
                player_washout_decision_array[i,j,k] = 1
                player_washout_movement_time[i,j,k] = q[0] - s[0]
                player_washout_reach_time[i,j,k] = q[0]
            elif np.size(r)>0: #if LH enters left target
                player_washout_decision_array[i,j,k] = -1   
                player_washout_movement_time[i,j,k] = r[0] - s[0]
                player_washout_reach_time[i,j,k] = r[0]
            if player_washout_reach_time[i,j,k] > 1500 or np.isnan(player_washout_reach_time[i,j,k]):
                player_washout_decision_array[i,j,k] = 0
            
        # #plot path for washouts
        if False:
            for i in range(1):
                plot_end_time = int(start_time + player_washout_reach_time[i,j,k])
                lhx_new = np.array(washout_df.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                lhy_new = np.array(washout_df.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                plt.figure(dpi=300)
                circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                fig, ax = plt.subplots()
                ax.add_patch(circleR)
                ax.add_patch(circleL)
                ax.add_patch(startCirc)
                plt.plot(lhx_new,lhy_new) 
                plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])

                plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                plt.show()

           

0 Sub1_Task


100%|██████████| 25/25 [00:00<00:00, 29.68it/s]


1 Sub2_Task


100%|██████████| 25/25 [00:01<00:00, 23.83it/s]


2 Sub3_Task


...

100%|██████████| 25/25 [00:00<00:00, 39.14it/s]


18 Sub19_Task


100%|██████████| 25/25 [00:01<00:00, 19.32it/s]


19 Sub20_Task


100%|██████████| 25/25 [00:02<00:00, 11.55it/s]


Pickle Washout

In [ ]:
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    pickle.dump(washout_data[i,:,:,:,:], open(data_path + f'{subname}_washout_data.pkl', 'wb'))
    pickle.dump(player_washout_decision_time[i,:], open(data_path + f'{subname}_player_washout_decision_time.pkl', 'wb'))
    pickle.dump(player_washout_decision_array[i,:], open(data_path + f'{subname}_player_washout_decision_array.pkl', 'wb'))
    pickle.dump(player_washout_movement_time[i,:], open(data_path + f'{subname}_player_washout_movement_time.pkl', 'wb'))
    pickle.dump(player_washout_reach_time[i,:], open(data_path + f'{subname}_player_washout_reach_time.pkl', 'wb'))
    pickle.dump(agent_washout_decision_time[i,:], open(data_path + f'{subname}_agent_washout_decision_time.pkl', 'wb'))
    pickle.dump(agent_washout_decision_array[i,:], open(data_path + f'{subname}_agent_washout_decision_array.pkl', 'wb'))
    pickle.dump(agent_washout_reach_time[i,:], open(data_path + f'{subname}_agent_washout_reach_time.pkl', 'wb'))

Sub1
Sub2
Sub3
Sub4
Sub5
Sub6
Sub7
Sub8
Sub9
Sub10
Sub11
Sub12
Sub13
Sub14
Sub15
Sub16
Sub17
Sub18
Sub19
Sub20


Keep Order That Participants Did Condition

In [ ]:
agent_task_decision_array[0]

array([[-1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,
        -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
         1., -1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,
         1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
         1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
         1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
        -1., -1.],
       [-1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
        -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
        -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
        -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,
         0.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,
         1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
         1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
        -1., 

Use event codes to get wins and indecisions

In [ ]:
# # Use event codes to find number of wins and indecisions
# #%% Extract Hand Position data
# for w in range(num_subjects): #len(pull_list)):
#     print(w)
#     sub_name = analysis_pull_list_task[w]
#     print(sub_name)
#     path1 = PATH+'\\'+sub_name
#     file = path1+f'\\{sub_name}Trial_Table.csv'
#     task_trial_table = pd.read_csv(file)
#     task_trial_table = task_trial_table.loc[task_trial_table['TP_Row']>=3]
#     for i in tqdm((range(tot_trials))):
#         block_number = task_trial_table.iloc[i]['Block_Row']
#         tp_num = task_trial_table.iloc[i]['TP_Row']
#         block_trial_num = task_trial_table.iloc[i]['Block_Step']
#         x = tp_num - 3 #used for indexing, subtraction number needs to change to 1 if I start with TP1 
#         c = block_trial_num - 1 #used for indexing, block trial number starts with 1
#         filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v3_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
#         filename_storage[w,x,c] = filename
#         data = pd.read_csv(filename, low_memory=False) #read in data
#         #Create start time and max end time array
#         score_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
#         trial_start[w,x,c]= int(data[data['Event_Codes']==list_event_codes[0]].index[0]) #if event codes column is equal to the list event codes, give the index (time)
#         trial_start = trial_start.astype(int)
#         end_time = trial_start[w,x,c]+2000
#         agent_target_select_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
#         # Get Agent mean reach time... data starts at next index from E_SOUNG_SIGNAL aka trial_start
#         agent_decision_time[w,x,c] = data.iloc[trial_start[w,x,c]+1]['Agent_Reach_Time'] #CALLED AGENT REACH TIME BUT IT'S NOW DECISION TIME IN V3
#         agent_decision_array[w,x,c] = data.iloc[agent_target_select_index]['Agent_decision_array']
#         agent_indecision_index = np.argwhere(agent_decision_time+150>1500)
#         for i,j,k in agent_indecision_index:
#             agent_decision_array[i,j,k] = 0
#         data = np.array(data)
#         event_codes[w,x,c,:] = data[trial_start[w,x,c]:end_time,4] # create event codes array
#         hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
#         hand_data = hand_data.astype(float)
#         hand_pos_data[w,x,c,:,:] = hand_data[trial_start[w,x,c]:end_time,:]
        
#         # If you want the scores
#         # scoreA = data[score_index, 4]
#         # scoreB = data[score_index, 5]
#         # if scoreA == 1:
#         #     win_check_R[w,x]+=1
#         # if scoreB == 1:
#         #     win_check_L[w,x]+=1
#         # print(filename)
#         # print(data[trial_start[w,x,c]:end_time,:])
#         #------------------------------------------------------------------
           
#         # LEFT HAND
#         lhx = hand_pos_data[w,x,c,:,2]
#         lhy = hand_pos_data[w,x,c,:,3]
#         q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_radius/2)
#         r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_radius/2) 
#         s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_radius/2)
#         if np.size(s)>0: 
#             player_decision_time[w,x,c] = s[0]
#         if np.size(q)>0: #if LH enters right target
#             player_decision_array[w,x,c] = 1
#             player_reach_time[w,x,c] = q[0]
#         elif np.size(r)>0: #if LH enters left target
#             player_decision_array[w,x,c] = -1   
#             player_reach_time[w,x,c] = r[0]
            
#         if player_reach_time[w,x,c] > 1500 or np.isnan(player_reach_time[w,x,c]):
#             player_decision_array[w,x,c] = 0